In [29]:
import cvxpy as cp
import numpy as np
from numpy.typing import NDArray

def get_ordered_comparison_model(line_array : NDArray, epsilon=1e-5, M = 1e5):
    ordered_list = get_list_of_arrays_less_than_k(line_array)
    Cis = cp.Variable(len(line_array), boolean=True)
    constraints_hi = []
    for idx in range(len(ordered_list)):
        constraints_hi += [ordered_list[idx][idx] >= ordered_list[k] + epsilon - M*(1-Cis[idx])
                          for k in range(len(ordered_list[idx])-1)]
    return Cis, constraints_hi

def get_list_of_arrays_less_than_k(line_array : NDArray):
    return [line_array[:k+1] for k in range(len(line_array))]

# Example data
A = np.array([10, 15, 12, 20,10, 40,41, 52, 53, 23, 22, 22]) 
print(get_list_of_arrays_less_than_k(A))
epsilon = 1e-5  # Tolerance for strict inequality
M = 1e5  # Big-M value

Cis, constraints = get_ordered_comparison_model(A, epsilon, M)
objective = cp.Maximize(cp.sum(Cis))
problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.SCIP)

print(problem.status)

# Results
print(f"Number of towers to see: {problem.value}")

[array([10]), array([10, 15]), array([10, 15, 12]), array([10, 15, 12, 20]), array([10, 15, 12, 20, 10]), array([10, 15, 12, 20, 10, 40]), array([10, 15, 12, 20, 10, 40, 41]), array([10, 15, 12, 20, 10, 40, 41, 52]), array([10, 15, 12, 20, 10, 40, 41, 52, 53]), array([10, 15, 12, 20, 10, 40, 41, 52, 53, 23]), array([10, 15, 12, 20, 10, 40, 41, 52, 53, 23, 22]), array([10, 15, 12, 20, 10, 40, 41, 52, 53, 23, 22, 22])]
optimal
Number of towers to see: 7.0


In [39]:
import cvxpy as cp

def make_discrete_decision_variables(array_shape, allowed_values):
    # Binary variables: z[i,j,k] = 1 if C[i,j] == allowed_values[k]
    z = cp.Variable( array_shape + tuple([len(allowed_values)]), boolean=True)
    constraints = []
    for i in range(array_shape[0]):
        for j in range(array_shape[1]):
            # Only one value can be chosen per C[i,j]
            constraints += [cp.sum(z[i, j, :]) == 1]
    return z, constraints

def guarantee_one_of_each_in_full_matrix(z_array, allowed_values):
    constraints = []
    # for each column (it is supposed to be switched like that)
    for row in range(z_array.shape[0]):
        # Each value v must appear at least once in C
        for v_idx, _ in enumerate(allowed_values):
            constraints += [cp.sum(z_array[row,:, v_idx]) >= 1]  # At least one occurrence of v
    
    # for each row (it is supposed to be switched like that)
    for column in range(z_array.shape[0]):
        # Each value v must appear at least once in C
        for v_idx, _ in enumerate(allowed_values):
            constraints += [cp.sum(z_array[:,column, v_idx]) >= 1]  # At least one occurrence of v
    return constraints

n = 4
allowed_values = [1, 2, 3, 4]  # Discrete set
decision_variables, constraints = make_discrete_decision_variables((n, n), allowed_values)
constraints += guarantee_one_of_each_in_full_matrix(decision_variables, allowed_values)

z = decision_variables

# fill some just to test
test_contraints = [z[0, 0, 0] == 1, z[1, 0, 2] == 1, z[0, 2, 1] == 1, z[1, 2, 3] == 1,
                   z[1, 1, 0] == 1 , z[3, 3, 3] == 1]
constraints += test_contraints

objective = cp.Maximize(cp.sum(decision_variables))

problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.SCIP)

print(problem.status)
#print(decision_variables.value)
decisions = np.array(decision_variables.value)
final_result = np.zeros(decisions[:,:,0].shape)

for idx in range(decisions.shape[2]):
    final_result += decisions[:,:, idx]*allowed_values[idx]

print(final_result)


optimal
[[1. 4. 2. 3.]
 [3. 1. 4. 2.]
 [4. 2. 3. 1.]
 [2. 3. 1. 4.]]


/home/pedro.trindade/anaconda3/envs/optmization/lib/python3.13/site-packages/cvxpy/reductions/solvers/solving_chain.py:418: UserWarning: The problem has an expression with dimension greater than 2. Defaulting to the SCIPY backend for canonicalization.
  warnings.warn(UserWarning(
